# Grain Coarsening

In [ ]:
import pyvista as pv

pv.set_jupyter_backend("static")

%load_ext autoreload
%autoreload 2

In [ ]:
from materialite import Material
import numpy as np
from materialite.models.grain_coarsening_model import (
    GrainCoarseningModel,
    calculate_potts_energy,
)

Materialite implements a "Grain Coarsening" model, which uses the kinetic Potts Monte Carlo method to simulate grain coarsening during heat treatment.
A grain coarsening model requires three input fields:
- A "grain" integer field which represents grains within a material. Historically, kinetic Potts Monte Carlo models have used the term "spin" for this quantity.
- A "temperature" field that represents a real or pseudo-temperature field.
- A "mobility" field that reflects how mobile or how likely individual sites are to flip.

The temperature and mobility fields are interrelated with one another. Temperature by itself does not impact the simulation. However, there is often a temperature-mobility relationship that relates the temperature to the mobility.
For now, we can assign them both to be one, create a grain field with values from 0 to 20, and plot it. 

In [ ]:
material = (
    Material(dimensions=[40, 40, 40])
    .create_random_integer_field("grain", 0, 20)
    .create_uniform_field("temperature", 1.0)
    .create_uniform_field("mobility", 1.0)
)
material.plot("grain")

Each site has been randomly assigned a grain value between 0 and 20. There are repeated values, so more than one site can share the same value. One way of thinking about it is by considering each grain id as corresponding to unique a crystallographic orientation.

The grain coarsening model takes various inputs, including the maximum integer value of the grains, and the number of flip attempts to try. When running the model, it needs to know what the grain field has been called. 

After running the model, the grain field can be visualized again. We see that the grains have been coarsened and it looks closer to an equiaxed microstructure.

In [ ]:
model = GrainCoarseningModel(
    max_grain_id=20,
    num_flip_attempts=3e5,
)


coarsened_material = model.run(material)


coarsened_material.plot("grain")

If more Monte Carlo flip attempts are taken, the structure will be further coarsened.

In [ ]:
bigger_model = GrainCoarseningModel(
    max_grain_id=20,
    num_flip_attempts=2e6,
)


extra_coarsened_material = bigger_model.run(material)


extra_coarsened_material.plot("grain")

The total Monte Carlo energy of the three states (the initial material, coarsened material, and extra coarsened material) can be compared. This typically decreases logarithmically with the number of Monte Carlo steps. 

In [ ]:
print("Energy before:", calculate_potts_energy(material, model))
print("Energy after coarsening:", calculate_potts_energy(coarsened_material, model))
print(
    "Energy after extra coarsening:",
    calculate_potts_energy(extra_coarsened_material, model),
)

The mobility field can be used to simulate phenomena such as pinning. 

In [ ]:
rng = np.random.default_rng(0)
pinned_points = rng.choice(material.num_points, 5000, replace=False)
pinned_field = material.extract("mobility")
pinned_field[pinned_points] = 0.0
pinned_material = material.create_fields({"mobility": pinned_field})
pinned_material.plot("mobility")

In [ ]:
extra_coarsened_pinned_material = bigger_model.run(pinned_material)
extra_coarsened_pinned_material.plot("grain")

In [ ]:
print(
    "Energy after coarsening with no pinning:",
    calculate_potts_energy(extra_coarsened_material, model),
)


print(
    "Energy after coarsening with pinning:",
    calculate_potts_energy(extra_coarsened_pinned_material, model),
)

Mobility and temperature can also be tied together by defining and passing in a temperature/mobility relationship. First, we create a temperature field that is linearly related to the z-position:

In [ ]:
z_coordinates = material.extract("z")
linear_temperature = z_coordinates / z_coordinates.max()
material = material.create_fields({"temperature": linear_temperature})
material.plot("temperature")

Now we define a relationship between temperature and mobility, pass it into a model, and run:

In [ ]:
def mobility_temperature_relationship(temperature):
    return temperature**2


temperature_model = GrainCoarseningModel(
    max_grain_id=20,
    num_flip_attempts=2e6,
    temperature_mobility_relationship=mobility_temperature_relationship,
)

In [ ]:
temperature_material = temperature_model.run(material)
temperature_material.plot("grain")

Mobility decreases to zero as it approaches the bottom of the domain, which causes minimal refinement in that portion of the material.